In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# create expression matrix

In [2]:
data_path = Path("/mnt/d/dailywork/Smear_seq/cancer/eso_squamou_cancer/TCGA_eso_data/gdc_download_20240926_065030.264552")

# create a empty list to record all data
df_list = []

for path in data_path.iterdir():
    path = [i for i in path.glob("*.tsv") if path.is_dir()][0]
    data = pd.read_csv(path, sep="\t", comment="#")
    # the information of first four lines is data summary, so filtering it
    data = data.iloc[4:]
    # check whether the list length of gene id and the list length of gene name is same
    if len(set(data.gene_name.tolist())) == len(set(data.gene_id)):
        print("hahahah")
    else:
        numeric_df = data.select_dtypes(include=['float', 'int'])
        df = numeric_df.groupby(data["gene_name"]).mean()
        df.reset_index(inplace=True)
        df.index.name = None

        df = df.pivot_table(index=None, columns='gene_name', values='fpkm_unstranded')
        
    index_value = path.name
    df.index = [index_value]
    df_list += [df]

In [3]:
expression_data = pd.concat(df_list, axis=0)

In [4]:
expression_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 708cd0e0-63bd-4728-b348-ee25ebfa98be.rna_seq.augmented_star_gene_counts.tsv to 023b5710-65a2-4990-a708-3f375df39241.rna_seq.augmented_star_gene_counts.tsv
Columns: 59427 entries, 5S_rRNA to snoZ196
dtypes: float64(59427)
memory usage: 44.4+ MB


In [5]:
expression_data.shape

(98, 59427)

# map file name to id in order to find survival time

In [6]:
with open("/mnt/d/dailywork/Smear_seq/cancer/eso_squamou_cancer/TCGA_eso_data/metadata.cart.2024-09-26.json", "r") as f:
    id_info = json.load(f)

id_df = pd.json_normalize(id_info)

In [7]:
id_df.head()

,data_format,access,associated_entities,file_name,submitter_id,data_category,platform,file_size,md5sum,file_id,...,analysis.workflow_version,analysis.input_files,analysis.updated_datetime,analysis.workflow_link,analysis.submitter_id,analysis.state,analysis.workflow_type,analysis.analysis_id,analysis.created_datetime,annotations
0,TSV,open,[{'entity_submitter_id': 'TCGA-IG-A97I-01A-11R...,efee1a58-1e28-43d2-92bf-37f8b9107312.rna_seq.a...,124bdacc-57bc-442e-b645-c9a9946609ee,Transcriptome Profiling,Illumina,4264013,f4f2c72cc8ae58c28bda7b6f03927c77,a36c99ec-753c-4859-9082-b077434a3f23,...,5d8c131bbff59fb0c969217fc1d44e6d1503cd1f,"[{'data_format': 'BAM', 'proportion_reads_mapp...",2022-01-19T12:30:05.251167-06:00,https://github.com/NCI-GDC/gdc-rnaseq-cwl/blob...,efee1a58-1e28-43d2-92bf-37f8b9107312_star__counts,released,STAR - Counts,3072a48a-1872-4fc7-bf38-43b1e8c27fb6,2021-12-13T16:50:23.375729-06:00,NaN
1,TSV,open,[{'entity_submitter_id': 'TCGA-IG-A3I8-01A-11R...,40ac3474-e908-43db-861b-98e9bfb61739.rna_seq.a...,f8c60df3-f92a-4b25-a2b0-f23c75bc5e28,Transcriptome Profiling,Illumina,4267675,cf963850521b74345ba4f9ee87a0317c,590af49d-2e1d-4cad-ab8c-9abe3c7558b0,...,5d8c131bbff59fb0c969217fc1d44e6d1503cd1f,"[{'data_format': 'BAM', 'proportion_reads_mapp...",2022-01-19T12:30:05.251167-06:00,https://github.com/NCI-GDC/gdc-rnaseq-cwl/blob...,40ac3474-e908-43db-861b-98e9bfb61739_star__counts,released,STAR - Counts,c354a314-36de-4ba2-8be8-19bfe0144af9,2021-12-13T16:49:34.804347-06:00,NaN
2,TSV,open,[{'entity_submitter_id': 'TCGA-IG-A3YB-01A-11R...,de4f9407-b1e1-4377-a106-a7cc3a1cbd95.rna_seq.a...,ddd99f56-d35e-4062-8f78-5da234083282,Transcriptome Profiling,Illumina,4252072,e66c7394c0e128c1d4ba652d5a7c3986,a5f873f0-50b6-44c4-8864-9651b3e26d8f,...,5d8c131bbff59fb0c969217fc1d44e6d1503cd1f,"[{'data_format': 'BAM', 'proportion_reads_mapp...",2022-01-19T12:30:05.251167-06:00,https://github.com/NCI-GDC/gdc-rnaseq-cwl/blob...,de4f9407-b1e1-4377-a106-a7cc3a1cbd95_star__counts,released,STAR - Counts,c4c414b9-ca4a-434c-a637-22ab92b8322c,2021-12-13T16:50:15.083340-06:00,NaN
3,TSV,open,[{'entity_submitter_id': 'TCGA-JY-A93F-01A-21R...,4fc22b47-3583-433a-abd3-73c10f555529.rna_seq.a...,d34b0397-5600-4fd3-bd1c-22ad8ea42c0f,Transcriptome Profiling,Illumina,4240208,8a703a2821c27e4aee2e713c0cb5a770,b744fd0f-cddf-4277-87da-7810d551dd42,...,5d8c131bbff59fb0c969217fc1d44e6d1503cd1f,"[{'data_format': 'BAM', 'proportion_reads_mapp...",2022-01-19T12:30:05.251167-06:00,https://github.com/NCI-GDC/gdc-rnaseq-cwl/blob...,4fc22b47-3583-433a-abd3-73c10f555529_star__counts,released,STAR - Counts,6ac669ae-c39a-4d7b-b443-9ebc738ef954,2021-12-13T16:47:53.495695-06:00,NaN
4,TSV,open,[{'entity_submitter_id': 'TCGA-LN-A4A5-01A-21R...,32ac2b37-8236-4738-a265-623d4c7932ad.rna_seq.a...,69c06144-ae6a-444a-acd1-891b2d122511,Transcriptome Profiling,Illumina,4267407,4a14636e3383ce579b4210612b4c5d46,b381948c-a679-4666-b1a4-87c84aec259b,...,5d8c131bbff59fb0c969217fc1d44e6d1503cd1f,"[{'data_format': 'BAM', 'proportion_reads_mapp...",2022-01-19T12:30:05.251167-06:00,https://github.com/NCI-GDC/gdc-rnaseq-cwl/blob...,32ac2b37-8236-4738-a265-623d4c7932ad_star__counts,released,STAR - Counts,45cc8619-1768-4729-b764-61027c781eac,2021-12-13T16:49:49.825164-06:00,NaN


In [8]:
file_id_dict = {file_name: id[0]['case_id'] for id, file_name in zip(id_df['associated_entities'], id_df['file_name'])}
file_id_df = pd.DataFrame.from_dict(file_id_dict, orient="index")
file_id_df.columns = ["id"]

In [9]:
# merge expression data with case_id
exp_with_id = pd.merge(expression_data, file_id_df, left_index=True, right_index=True)

In [10]:
exp_with_id.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 708cd0e0-63bd-4728-b348-ee25ebfa98be.rna_seq.augmented_star_gene_counts.tsv to 023b5710-65a2-4990-a708-3f375df39241.rna_seq.augmented_star_gene_counts.tsv
Columns: 59428 entries, 5S_rRNA to id
dtypes: float64(59427), object(1)
memory usage: 44.4+ MB


# survival data

In [11]:
sur_data = pd.read_csv("/mnt/d/dailywork/Smear_seq/cancer/eso_squamou_cancer/TCGA_eso_data/overall-survival-plot.2024-09-26.tsv", sep="\t")

In [12]:
sur_data = sur_data[["id", "time", "censored"]]

In [16]:
sur_data.columns = ["id", "time", "overall_survival"]
sur_data['overall_survival'] = [1 if _ else 0 for _ in sur_data['overall_survival']]
sur_data.head()

,id,time,overall_survival
0,cad8bd3a-ffd8-4baf-9de6-3ac3a1eb19e1,0.010951,1
1,06ca7fc4-4ae0-47d5-9e83-f92ba42a957b,0.024641,0
2,9e54a27f-d0fa-4a5b-91bc-13e27fed2a3d,0.043806,1
3,a4d36e09-5d8e-4006-855c-e466682b6813,0.065708,0
4,4b8809a6-4ab3-4567-8623-37ce3faa978c,0.071184,0


In [17]:
final_df = pd.merge(exp_with_id, sur_data, on="id")

In [18]:
final_df.head()

,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,ZYX,ZYXP1,ZZEF1,ZZZ3,hsa-mir-1253,hsa-mir-423,snoZ196,id,time,overall_survival
0,0.013556,0.042650,0.012400,0.0129,0.1188,0.0148,28.8769,0.1492,83.9359,0.9729,...,36.2874,0.0,7.9550,3.2894,0.0,0.0,0.0000,4d4e15d1-5d54-4ee3-808f-0b28dfc77871,0.616016,1
1,0.015967,0.017733,0.019386,0.0384,0.2756,0.0025,98.5178,0.5706,2.1618,0.0179,...,69.5627,0.0,9.1938,6.3909,0.0,0.0,0.0000,ae404398-6544-4f3a-8830-44a4a227f8d5,1.034908,1
2,0.053822,0.050550,0.012200,0.0404,0.3622,0.0012,52.7419,0.1877,2.4358,0.0000,...,109.1493,0.0,4.4254,4.8772,0.0,0.0,0.2591,b997d486-8c71-4926-9df1-d2afdaf2785f,0.284736,1
3,0.011889,0.074767,0.000000,0.0284,0.2924,0.0071,27.3096,0.3154,10.1859,0.0251,...,76.2110,0.0,2.9933,4.3215,0.0,0.0,0.0000,0ac08ecf-4d70-48a4-9dfd-f44032551cb9,2.932238,1
4,0.076133,0.000000,0.011814,0.0277,0.0256,0.0026,18.2233,0.0962,404.3192,0.9275,...,23.1724,0.0,5.8513,2.9127,0.0,0.0,0.1385,ef573329-9659-4106-b391-9a33e1c732c4,0.522930,1


In [19]:
final_df.to_csv("/mnt/d/dailywork/Smear_seq/cancer/eso_squamou_cancer/TCGA_eso_data/gene_expression_with_survival_time.csv", index=None)